In [4]:
import os
import json
import requests
import geojsonio
import time
import geopandas as gpd
import geojson
from os.path import dirname as up
import pandas as pd

# Helper function to printformatted JSON using the json module
def p(data):
    print(json.dumps(data, indent=2))


In [5]:
root_path = up(os.getcwd())
geom_path = os.path.join(root_path, 'data/bounding1.geojson')

with open(geom_path) as f:
    geom = geojson.load(f)

features = geom['features'][0]
test_geom = features['geometry']

In [6]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if os.environ.get('PL_API_KEY', ''):
    API_KEY = os.environ.get('PL_API_KEY', '')
else:
    API_KEY = 'PLAK0cf1acfbb7514297901e134a1e09efcb'

# construct auth tuple for use in the requests library
BASIC_AUTH = (API_KEY, '')

# Setup Planet Data API base URL
URL = "https://api.planet.com/data/v1"

# Setup the session
session = requests.Session()

# Authenticate
session.auth = (API_KEY, "")

# Make a GET request to the Planet Data API
res = session.get(URL)

# Response status code
res.status_code

200

In [119]:
# Setup the stats URL
stats_url = "{}/stats".format(URL)

# Print the stats URL
print(stats_url)

https://api.planet.com/data/v1/stats


In [7]:
# Create filter object for all imagery captured after 2019, 
# and within the given geometry, and with the instrument types

# Setup Date Filter
date_filter = {
    "type": "DateRangeFilter", # Type of filter -> Date Range
    "field_name": "acquired", # The field to filter on: "acquired" -> Date on which the "image was taken"
    "config": {
        "gte": "2021-01-01T00:00:00.000Z", # "gte" -> Greater than or equal to
        "lte": "2024-03-01T00:00:00.000Z"
    }
}


# filter any images which are more than 50% clouds
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.0
  }
}


# Setup Geometry Filter
geometry_filter = {
    "type": "GeometryFilter",
    "field_name": "geometry",
    "config": test_geom
}


# permission_filter = {
#     "type": "PermissionFilter",
#     "config": ["assets.analytic:download"]
# }


# Search for imagery only from PlanetScope satellites that have a PS2 telescope
# Setup item types
item_types = ["PSScene"] #item_types = ["PSScene", "REOrthoTile"] 

# Setup a filter for instrument type
# Instrument info: # https://developers.planet.com/docs/data/psscene/
instrument_filter = {
    "type": "StringInFilter",
    "field_name": "instrument",
    "config": ["PSB.SD"] #, "PS2"
}

# Setup an "AND" logical filter
and_filter = {
    "type": "AndFilter",
    "config": [instrument_filter, geometry_filter, date_filter, cloud_cover_filter]
}

# # Print the logical filter
# p(and_filter)

In [121]:
# Setup the request
request = {
    "item_types" : item_types,
    "interval" : "year",
    "filter" : and_filter
}

# Send the POST request to the API stats endpoint
res=session.post(stats_url, json=request)

# Print response
p(res.json())

{
  "buckets": [
    {
      "count": 64,
      "start_time": "2021-01-01T00:00:00.000000Z"
    },
    {
      "count": 96,
      "start_time": "2022-01-01T00:00:00.000000Z"
    },
    {
      "count": 123,
      "start_time": "2023-01-01T00:00:00.000000Z"
    },
    {
      "count": 15,
      "start_time": "2024-01-01T00:00:00.000000Z"
    }
  ],
  "interval": "year",
  "utc_offset": "+0h"
}


In [91]:
# organize the request by removing 0 count image

def filterOut(injson):
    
    allsource = injson['buckets']
    validsource = list()
    
    for i in range(0,len(allsource)):
        if allsource[i]["count"] != 0:
            validsource.append(allsource[i])
        else:
            pass
    
    return validsource

validcount = filterOut(res.json())
validcount

[{'count': 64, 'start_time': '2021-01-01T00:00:00.000000Z'},
 {'count': 96, 'start_time': '2022-01-01T00:00:00.000000Z'},
 {'count': 123, 'start_time': '2023-01-01T00:00:00.000000Z'},
 {'count': 15, 'start_time': '2024-01-01T00:00:00.000000Z'}]

## Get all page links

In [19]:
import json
import requests
from requests.auth import HTTPBasicAuth

item_type = "PSScene"

# API request object
search_request = {
  "item_types": item_types, 
  "filter": and_filter
}

# fire off the POST request
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(API_KEY, ''),
    json=search_request)


geojson = search_result.json()
all_pages = list()

while geojson['_links']['_next']!=None: # when it's not to the final page
        
    all_pages.append(geojson['_links']['_self']) # adding the first page link
        
    next_page = geojson['_links']['_next']

    next_info = session.get(next_page)
    geojson = next_info.json()


# # let's look at the first result
# print(list(geojson.items())[1][1][0])

In [39]:
df = pd.DataFrame()

for link in all_pages:

    page_info = session.get(link).json()
    sub_df = pd.DataFrame(page_info['features'])
    df = df.append(sub_df)

image_ids = df['id'].tolist()

In [41]:
# For demo purposes, just grab the first image ID
id0 = image_ids[0]
id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, id0) # 8bands type

# Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
result = \
  requests.get(
    id0_url,
    auth=HTTPBasicAuth(API_KEY, '')
  )

# List of asset types available for this particular satellite image
print(result.json().keys())

dict_keys(['basic_analytic_4b', 'basic_analytic_4b_rpc', 'basic_analytic_4b_xml', 'basic_analytic_8b', 'basic_analytic_8b_xml', 'basic_udm2', 'ortho_analytic_4b', 'ortho_analytic_4b_sr', 'ortho_analytic_4b_xml', 'ortho_analytic_8b', 'ortho_analytic_8b_sr', 'ortho_analytic_8b_xml', 'ortho_udm2', 'ortho_visual'])


In [43]:
# This is "inactive" if the "ortho_analytic_4b" asset has not yet been activated; otherwise 'active'
print(result.json()['ortho_analytic_8b']['status'])

inactive


In [46]:
# Parse out useful links
links = result.json()[u"ortho_analytic_8b"]["_links"]
self_link = links["_self"]
activation_link = links["activate"]

# Request activation of the 'ortho_analytic_4b' asset:
activate_result = \
  requests.get(
    activation_link,
    auth=HTTPBasicAuth(API_KEY, '')
  )

In [47]:
activation_status_result = \
  requests.get(
    self_link,
    auth=HTTPBasicAuth(API_KEY, '')
  )
    
print(activation_status_result.json()["status"])

activating


In [48]:

def activating_asset(sid, itemType, productKey):
    
    id_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(itemType, sid)
    # Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
    result = \
      requests.get(
        id0_url,
        auth=HTTPBasicAuth(API_KEY, '')
      )

    # Parse out useful links
    links = result.json()[productKey]["_links"]
    self_link = links["_self"]
    activation_link = links["activate"]

    # Request activation of the asset:
    activate_result = \
      requests.get(
        activation_link,
        auth=HTTPBasicAuth(API_KEY, '')
      )
    
    activation_status_result = \
      requests.get(
        self_link,
        auth=HTTPBasicAuth(API_KEY, '')
      )

    print(activation_status_result.json()["status"])


In [49]:
activating_asset(image_ids[0], item_types[0], 'ortho_analytic_8b')

activating


In [51]:
allids = df.id.tolist()
item_type = "PSScene"
instruKey = "ortho_analytic_8b"

for img_id in image_ids:
    
    id_status = activating_asset(img_id, item_types[0], 'ortho_analytic_8b')
    
    if id_status == 'active':
        pl_download(img_id, item_type, instruKey, dataDir, filename=None)
    else:
        print('Scene id {} is not active'.format(sID))

active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active
active

In [11]:
def get_pages(session, API_KEY):
    
    all_pages = list()
    
    # fire off the POST request
    search_result = \
      requests.post(
        'https://api.planet.com/data/v1/quick-search',
        auth=HTTPBasicAuth(API_KEY, ''),
        json=search_request)

    geojson = search_result.json()
    
    
    while geojson['_links']['_self'] != geojson['_links']['_next']: # when it's not to the final page
        
        all_pages.append(geojson['_links']['_self']) # adding the first page link
        
        next_page = geojson['_links']['_next']
        next_info = session.get(next_page)
        geojson = next_info.json()
    
    all_pages.append(geojson['_links']['_self'])
    
    return all_pages

## Quick search

In [26]:
# Setup the quick search endpoint url
quick_url = "{}/quick-search".format(URL)

# Setup the request
request = {
    "item_types" : item_types,
    "filter" : and_filter
}

# Send the POST request to the API quick search endpoint
res = session.post(quick_url, json=request)

# # Assign the response to a variable
geojson = res.json()

# # # Print the response
# p(geojson)

In [27]:
# Print the response "_links" property
p(geojson["_links"])

{
  "_first": "https://api.planet.com/data/v1/searches/08f589600725409aa83b86c6b675ebe1/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9wcmV2IjogZmFsc2UsICJxdWVyeV9wYXJhbXMiOiB7fX0%3D",
  "_next": "https://api.planet.com/data/v1/searches/08f589600725409aa83b86c6b675ebe1/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6ICIyMDIxLTA3LTI1VDExOjI0OjM0LjAwMDAwMFoiLCAic29ydF9sYXN0X2lkIjogIjIwMjEwNzI0XzIzMTA0MV8yN18yNDYzIiwgInNvcnRfcHJldiI6IGZhbHNlLCAicXVlcnlfcGFyYW1zIjoge319",
  "_self": "https://api.planet.com/data/v1/searches/08f589600725409aa83b86c6b675ebe1/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9wcmV2IjogZmFsc2UsICJxdWVyeV9wYXJhbXMiOiB7fX0%3D"
}


In [28]:
# Assign the "_links" -> "_next" property (link to next page of results) to a variable 
next_url = geojson["_links"]["_next"]

# Print the link to the next page of results
print(next_url)

https://api.planet.com/data/v1/searches/08f589600725409aa83b86c6b675ebe1/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6ICIyMDIxLTA3LTI1VDExOjI0OjM0LjAwMDAwMFoiLCAic29ydF9sYXN0X2lkIjogIjIwMjEwNzI0XzIzMTA0MV8yN18yNDYzIiwgInNvcnRfcHJldiI6IGZhbHNlLCAicXVlcnlfcGFyYW1zIjoge319


In [29]:
# Send the POST request to the API quick search endpoint with a page size of 9
res = session.post(quick_url, json=request, params={"_page_size" : 10})

# Assign the response to a variable
geojson = res.json()

# Get the number of features present in the response
len(geojson["features"])

10

In [30]:
# Assign the url variable to display the geojsonio map
url = geojsonio.display(res.text)

In [31]:
# Assign the next_url variable to the next page of results from the response (Setup the next page of results)
next_url = geojson["_links"]["_next"]

# Get the next page of results
res = session.get(next_url)

# Assign the response to a variable
geojson = res.json()

# Get the url see results on geojson.io
url = geojsonio.to_geojsonio(res.text)

# Get the feature information

In [44]:
# Assign a features variable 
features = geojson["features"]

# Get the number of features present in the response
len(features)

TypeError: 'module' object is not subscriptable

In [ ]:
def exportMeta(feat):
    
    metadf = pd.DataFrame([feat['properties']])
    metadf['id'] = feat['id']
    
    return metadf
    
    

In [ ]:
for i in range(0, len(features)):
    
    if i == 0:
        df = exportMeta(features[i])
    else:
        tempdf = exportMeta(features[i])
        df = pd.concat([df, tempdf])

df = df.sort_values(by=['acquired'])
df['date'] = df['id'].apply(lambda x: x.split('_')[0])
df['hour_min_sec'] = df['id'].apply(lambda x: x.split('_')[1])
df.to_csv('../data/processing_data/test.csv', index=False)

# Activate data for downloding

In [ ]:
# extract image IDs only
image_ids = [feature['id'] for feature in geojson['features']]
print(image_ids)

In [11]:
def activating_asset(sid, itemType, productKey):
    
    id_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(itemType, sid)
    # Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
    result = \
      requests.get(
        id0_url,
        auth=HTTPBasicAuth(API_KEY, '')
      )

    # Parse out useful links
    links = result.json()[productKey]["_links"]
    self_link = links["_self"]
    activation_link = links["activate"]

    # Request activation of the asset:
    activate_result = \
      requests.get(
        activation_link,
        auth=HTTPBasicAuth(API_KEY, '')
      )
    
    activation_status_result = \
      requests.get(
        self_link,
        auth=HTTPBasicAuth(API_KEY, '')
      )

    print(activation_status_result.json()["status"])


In [ ]:
activating_asset(image_ids[1], item_types[0], 'ortho_analytic_8b')

In [ ]:
import json
import requests
from requests.auth import HTTPBasicAuth

# For demo purposes, just grab the first image ID
id0 = image_ids[1]
id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_types[0], id0)

# Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
result = \
  requests.get(
    id0_url,
    auth=HTTPBasicAuth(API_KEY, '')
  )

# List of asset types available for this particular satellite image
print(result.json().keys())


In [ ]:
# This is "inactive" if the "ortho_analytic_4b" asset has not yet been activated; otherwise 'active'
print(result.json()['ortho_analytic_8b']['status'])

In [ ]:
# Parse out useful links
links = result.json()[u"ortho_analytic_8b"]["_links"]
self_link = links["_self"]
activation_link = links["activate"]

# Request activation of the 'ortho_analytic_4b' asset:
activate_result = \
  requests.get(
    activation_link,
    auth=HTTPBasicAuth(API_KEY, '')
  )

In [ ]:
activation_status_result = \
  requests.get(
    self_link,
    auth=HTTPBasicAuth(API_KEY, '')
  )
    
print(activation_status_result.json()["status"])

In [ ]:
# Image can be downloaded by making a GET with your Planet API key, from here:
download_link = activation_status_result.json()["location"]
print(download_link)

In [14]:
# Assign a variable to the search result features (items)
features = geojson["features"]

# Get the first result's feature
feature = features[0]

# Print the ID
p(feature["id"])

# Print the permissions
p(feature["_permissions"])

"20230124_151413_43_2262"
[
  "assets.basic_analytic_4b:download",
  "assets.basic_analytic_4b_rpc:download",
  "assets.basic_analytic_4b_xml:download",
  "assets.basic_analytic_8b:download",
  "assets.basic_analytic_8b_xml:download",
  "assets.basic_udm2:download",
  "assets.ortho_analytic_4b:download",
  "assets.ortho_analytic_4b_sr:download",
  "assets.ortho_analytic_4b_xml:download",
  "assets.ortho_analytic_8b:download",
  "assets.ortho_analytic_8b_sr:download",
  "assets.ortho_analytic_8b_xml:download",
  "assets.ortho_udm2:download",
  "assets.ortho_visual:download"
]


In [16]:
# Get the assets link for the item
assets_url = feature["_links"]["assets"]

# Print the assets link
print(assets_url)

https://api.planet.com/data/v1/item-types/PSScene/items/20230124_151413_43_2262/assets/


In [17]:
# Send a GET request to the assets url for the item (Get the list of available assets for the item)
res = session.get(assets_url)

# Assign a variable to the response
assets = res.json()

In [ ]:
# Print the asset types that are available
print(assets.keys())

In [ ]:
# Assign a variable to the visual asset from the item's assets
ortho_analytic8b = assets["ortho_analytic_8b_sr"]

# Print the visual asset data
p(ortho_analytic8b)

In [ ]:
# Setup the activation url for a particular asset (in this case the basic_analytic_4b asset)
activation_url = ortho_analytic8b["_links"]["activate"]

# Send a request to the activation url to activate the item
res = session.get(activation_url)

# Print the response from the activation request
p(res.status_code)

In [ ]:
asset_activated = False

while asset_activated == False:
    # Send a request to the item's assets url
    res = session.get(assets_url)

    # Assign a variable to the item's assets url response
    assets = res.json()

    # Assign a variable to the basic_analytic_4b asset from the response
    visual = assets["ortho_analytic_8b_sr"]

    asset_status = ortho_analytic8b["status"]
    print(asset_status)
    
    # If asset is already active, we are done
    if asset_status == 'active':
        asset_activated = True
        print("Asset is active and ready to download")

# Print the asset data    
p(ortho_analytic8b)

# Downloading

In [ ]:
# Assign a variable to the visual asset's location endpoint
location_url = ortho_analytic8b["location"]

# Print the location endpoint
print(location_url)

In [ ]:
# Create a function to download asset files
# Parameters: 
# - url (the location url)
# - filename (the filename to save it as. defaults to whatever the file is called originally)

def pl_download(url, filename=None):
    
    # Send a GET request to the provided location url, using your API Key for authentication
    res = requests.get(url, stream=True, auth=(API_KEY, ""))
    # If no filename argument is given
    if not filename:
        # Construct a filename from the API response
        if "content-disposition" in res.headers:
            filename = res.headers["content-disposition"].split("filename=")[-1].strip("'\"")
        # Construct a filename from the location url
        else:
            filename = url.split("=")[1][:10]
    # Save the file
    with open('output/' + filename, "wb") as f:
        for chunk in res.iter_content(chunk_size=1024):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                f.flush()

    return filename

# Download the file from an activated asset's location url
# pl_download(location_url)